In [138]:
import os
import json
import scipy.io as sio
import numpy as np

In [ ]:
'''
def get_json(demos, feas, bp, ext = 'train_cas'):
    final_data = [] # json
    I = "You are a personalized healthcare agent trained to predict systolic blood pressure (SBP) and diastolic blood pressure (DBP) based on user information and physiological data. Please note that SBP is typically higher than DBP, with SBP ranging from 85 to 180 mmHg and DBP ranging from 40 to 90 mmHg."
    for i in range(feas.shape[0]):
        if demos[i,2] == 0:
            gender = 'female'
        else:
            gender = 'male'

        Q1 = "The user's profile as age: {}-year-old, gender: {}, height: {:.1f} cm, weight: {:.1f} kg, history of hypertension: {}.".format(int(demos[i,1]),gender, demos[i,3],demos[i,4],int(demos[i,0]))
        ff = feas[i,:]
        Q2 = " The recent sensor readings show: {}. What would be the predicted SBP and DBP values?".format([round(x,2) for x in ff])
        A = "The predicted SBP is {:.1f} mmHg, and the predicted DBP is {:.1f} mmHg".format(bp[i,1],bp[i,0])

        
        final_data.append({'instruction':I, 'input':Q1+Q2, 'output':A})

        
    final_data = json.dumps(final_data, indent=4)
    with open("{}.json".format(ext), "w") as outfile:
        outfile.write(final_data)

get_json(trainDemo, trainSig, trainY, ext = 'train_cas_calibrationfree')
get_json(testDemo, testSig, testY, ext = 'test_cas_calibrationfree')
'''

In [139]:
def find_str(str):
  start_index = str.find("Based on this data")
  return str[0:start_index]

In [140]:
primer = " The output should be provided in this format: Predicted_map: <MAP_VALUE> mmHg, Predicted_pp: <PP_VALUE> mmHg."

filenamelist = ["cas_testlist1","cas_testlist2","cas_testlist3","cas_testlist4","cas_testlist5",
"cas_trainlist1","cas_trainlist2","cas_trainlist3","cas_trainlist4","cas_trainlist5"]

for filename in filenamelist:
    with open("./dataset/"+filename+".json","r",encoding="utf8") as f:
        data = json.load(f)
        changed_data=[]
        for l in data:
            inp = l["input"]
            map = round((float(l["refsbp"]) + float(l["refdbp"])*2)/3,1)
            pp = round(float(l["refsbp"]) - float(l["refdbp"]),1)
            output = "Predicted_map: {:.1f} mmHg, Predicted_pp: {:.1f} mmHg.".format(map,pp)

            changed_data.append({'instruction':l["instruction"], 'input':l["input"]+primer, "output": output, "refsbp":l["refsbp"], "refdbp":l["refdbp"], "basesbp":l["basesbp"], "basedbp":l["basedbp"]})
    #写到json中
    changed_data = json.dumps(changed_data, indent=4)
    with open("./dataset/"+"{}.json".format(filename+"_v2"), "w") as outfile:
        outfile.write(changed_data)
